# Projeto 2

**Disciplina:** C209-L4 – Computação Gráfica e Multimídia

**Aluno:** _Matheus Vieira Honório de Souza_

**Matrícula:** _525_  

---

## Descrição do Projeto
Este projeto visa aplicar técnicas de computação gráfica e edição de vídeo para transformar uma gravação original em um vídeo final com as seguintes etapas:
1. **Gravação do vídeo** lendo o texto fornecido.
2. **Inversão da imagem** no eixo (horizontal ou vertical) a cada 20 segundos.
3. **Redução gradual do volume**, de modo que os últimos 10 segundos fiquem completamente silenciosos.
4. **Corte do vídeo no segundo 60**, pulando direto para o segundo 80.
5. **Inserção dos 20 segundos cortados** (60 a 80s) ao final do vídeo.

## Plano de Implementação
1. Carregar o vídeo original usando `moviepy.editor.VideoFileClip`.
2. Aplicar inversão horizontal a cada 20 segundos alternadamente, utilizando `fl` com `np.fliplr` nos frames via `fl_image`.
3. Reduzir gradualmente o volume a cada 30 segundos com `audio.fl`, definindo um fator de volume que diminui ao longo do tempo e zera nos últimos 10 segundos.
3. Realizar corte temporal: manter o trecho de 0 a 60s, pular para 80s e seguir até o fim. Separar o trecho de 60s a 80s para ser reaproveitado.
4. Reorganizar os trechos com `concatenate_videoclips`, unindo: 0-60s + 80-fim + 60-80s.
5. Exportar o vídeo final com `write_videofile`, definindo codec `(libx264)` e audio_codec `(aac)`.


## Instalação da Versão Correta da Biblioteca

In [3]:
# Execute se necessário para garantir a versão correta
!pip install moviepy==1.0.3
!pip install numpy

## Bibliotecas Utilizadas

In [12]:
import moviepy.editor as mp
import numpy as np

## Carregamento do Vídeo Original

In [13]:
video = mp.VideoFileClip("video_original.mp4")
video.duration  # duração do vídeo em segundos

184.88

## Inversão da Imagem a Cada 20 Segundos (com NumPy)

In [14]:
def inverter_a_cada_20s(video):
    def processar_frame(get_frame, t):
        frame = get_frame(t)
        if int(t // 20) % 2 == 1:
            return np.fliplr(frame)  # Inversão horizontal com NumPy
        return frame

    return video.fl(processar_frame)

## Redução Gradual do Áudio

In [15]:
def reduzir_audio(video):
    audio = video.audio

    def processar_audio(gf, t):
        t = np.asarray(t)
        fator = np.ones_like(t)

        # volume decresce a cada 30s até zero nos últimos 10s
        fator[t >= video.duration - 10] = 0
        decaimento = (t // 30) * 0.1
        fator = np.clip(1 - decaimento, 0, 1) * fator

        return gf(t) * fator[:, np.newaxis]  # para áudio com múltiplos canais

    novo_audio = audio.fl(processar_audio)
    return video.set_audio(novo_audio)

## Corte e Rearranjo

In [16]:
def reordenar(video):
    parte1 = video.subclip(0, 60)
    parte2 = video.subclip(80, video.duration)
    parte3 = video.subclip(60, 80)
    return mp.concatenate_videoclips([parte1, parte2, parte3])

## Aplicando as Transformações

In [17]:
video_transformado = inverter_a_cada_20s(video)
video_transformado = reduzir_audio(video_transformado)
video_final = reordenar(video_transformado)

## Exportando o Vídeo Final

In [18]:
video_final.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
